In [1]:
import pandas as pd
import requests
from datetime import datetime
import pytz

In [2]:
#city name and url
city = 'Berlin'
API_key = 'APIKEY'

# check out the docs for more info on making an api call https://openweathermap.org/forecast5
url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")

response = requests.get(url)
response.status_code

200

In [3]:
json = response.json()
json

{'city': {'coord': {'lat': 52.5244, 'lon': 13.4105},
  'country': 'DE',
  'id': 2950159,
  'name': 'Berlin',
  'population': 1000000,
  'sunrise': 1657248804,
  'sunset': 1657308539,
  'timezone': 7200},
 'cnt': 40,
 'cod': '200',
 'list': [{'clouds': {'all': 75},
   'dt': 1657281600,
   'dt_txt': '2022-07-08 12:00:00',
   'main': {'feels_like': 20.23,
    'grnd_level': 1019,
    'humidity': 64,
    'pressure': 1009,
    'sea_level': 1009,
    'temp': 20.46,
    'temp_kf': -1.97,
    'temp_max': 22.43,
    'temp_min': 20.46},
   'pop': 0,
   'sys': {'pod': 'd'},
   'visibility': 10000,
   'weather': [{'description': 'broken clouds',
     'icon': '04d',
     'id': 803,
     'main': 'Clouds'}],
   'wind': {'deg': 308, 'gust': 8.35, 'speed': 6.58}},
  {'clouds': {'all': 83},
   'dt': 1657292400,
   'dt_txt': '2022-07-08 15:00:00',
   'main': {'feels_like': 19.59,
    'grnd_level': 1020,
    'humidity': 65,
    'pressure': 1014,
    'sea_level': 1014,
    'temp': 19.85,
    'temp_kf': 1.23

In [4]:
# for city info Tejal

#country = response['city']['country']
#city_name = response['city']['name']

#for weather info

#weather = response['list'][0]['main']['temp']
#temp = response['list'][0]['main']
#description = response['list'][0]['weather']
#weather

In [5]:
list1 = json['list'][0]
list1

{'clouds': {'all': 75},
 'dt': 1657281600,
 'dt_txt': '2022-07-08 12:00:00',
 'main': {'feels_like': 20.23,
  'grnd_level': 1019,
  'humidity': 64,
  'pressure': 1009,
  'sea_level': 1009,
  'temp': 20.46,
  'temp_kf': -1.97,
  'temp_max': 22.43,
  'temp_min': 20.46},
 'pop': 0,
 'sys': {'pod': 'd'},
 'visibility': 10000,
 'weather': [{'description': 'broken clouds',
   'icon': '04d',
   'id': 803,
   'main': 'Clouds'}],
 'wind': {'deg': 308, 'gust': 8.35, 'speed': 6.58}}

Now we've discovered what information we have to work with. Let's decide what we want to keep and what we wish to lose.

I feel that from json['list'] it would be good to keep

'weather.main', 'weather.description', 'dt_txt', 'main.temp', 'main.feels_like' 'clouds.all', 'rain.3h', 'snow.3h' 'wind.speed', 'wind.deg', 'main.humidity', 'main.pressure'.
And from json['city'] it would be good to keep

'name' and 'country.
Just to make sure that we got the right place. And as an added extra we'll also include the time that API call was made, so we know how up to date our forecast is.

Optional: let's get a timestamp of when we get the data. Datetime uses the uses the current time of the system, which on local computers is normally correct. But as we're in the cloud, computers are not always in our country, and we therefore add on the timezone module to ensure that our timestamp is local to us and not the computer.

In [6]:
tz = pytz.timezone('Europe/Berlin')
now = datetime.now().astimezone(tz)

now

datetime.datetime(2022, 7, 8, 13, 7, 17, 151069, tzinfo=<DstTzInfo 'Europe/Berlin' CEST+2:00:00 DST>)

In [7]:
# we'll store the information in this dicitonary:
weather_dict = {'city': [],
                'country': [],
                'forecast_time': [],
                'outlook': [],
                'detailed_outlook': [],
                'temperature': [],
                'temperature_feels_like': [],
                'clouds': [],
                'rain': [],
                'snow': [],
                'wind_speed': [],
                'wind_deg': [],
                'humidity': [],
                'pressure': []}
                #'information_retrieved_at': []}

# let's begin the loop
for i in json['list']:
    weather_dict['city'].append(json['city']['name'])  #city and country is out of list thats why we are using json file i.e go to json file and serach it
    weather_dict['country'].append(json['city']['country'])
    weather_dict['forecast_time'].append(i['dt_txt'])
    weather_dict['outlook'].append(i['weather'][0]['main'])
    weather_dict['detailed_outlook'].append(i['weather'][0]['description'])
    weather_dict['temperature'].append(i['main']['temp'])
    weather_dict['temperature_feels_like'].append(i['main']['feels_like'])
    weather_dict['clouds'].append(i['clouds']['all'])
    # sometimes the data is missing for rain and snow. As it is not always raining or snowing
    # we cannot make a DataFrame unless the lists are all the same length, therefore missing values are bad
    # here we say try to append a value if there is one. If not, append a 0
    try:
        weather_dict['rain'].append(i['rain']['3h'])
    except:
        weather_dict['rain'].append('0')
    try:
        weather_dict['snow'].append(i['snow']['3h'])
    except:
        weather_dict['snow'].append('0')
    weather_dict['wind_speed'].append(i['wind']['speed'])
    weather_dict['wind_deg'].append(i['wind']['deg'])
    weather_dict['humidity'].append(i['main']['humidity'])
    weather_dict['pressure'].append(i['main']['pressure'])
    #weather_dict['information_retrieved_at'].append(now.strftime("%d/%m/%Y %H:%M:%S"))
  

In [8]:
weather_from_dict_df = pd.DataFrame(weather_dict)

weather_from_dict_df

,city,country,forecast_time,outlook,detailed_outlook,temperature,temperature_feels_like,clouds,rain,snow,wind_speed,wind_deg,humidity,pressure
0,Berlin,DE,2022-07-08 12:00:00,Clouds,broken clouds,20.46,20.23,75,0,0,6.58,308,64,1009
1,Berlin,DE,2022-07-08 15:00:00,Clouds,broken clouds,19.85,19.59,83,0,0,4.67,309,65,1014
2,Berlin,DE,2022-07-08 18:00:00,Clouds,broken clouds,19.91,19.55,80,0,0,4.20,264,61,1019
3,Berlin,DE,2022-07-08 21:00:00,Clouds,scattered clouds,16.46,15.88,49,0,0,3.47,295,66,1024
4,Berlin,DE,2022-07-09 00:00:00,Clouds,scattered clouds,13.67,13.13,39,0,0,3.68,290,78,1024
5,Berlin,DE,2022-07-09 03:00:00,Clouds,scattered clouds,11.75,11.33,27,0,0,3.57,285,90,1024
6,Berlin,DE,2022-07-09 06:00:00,Clouds,scattered clouds,13.94,13.50,47,0,0,3.74,265,81,1023
7,Berlin,DE,2022-07-09 09:00:00,Clouds,overcast clouds,17.50,17.08,100,0,0,5.36,270,68,1022
8,Berlin,DE,2022-07-09 12:00:00,Rain,light rain,18.63,18.35,100,0.81,0,5.37,286,69,1020
9,Berlin,DE,2022-07-09 15:00:00,Rain,light rain,20.69,20.25,70,1.82,0,5.17,284,55,1019


In [9]:
def get_weather_loop(cities):
    API_key = 'APIKEY'

    tz = pytz.timezone('Europe/Berlin')
    now = datetime.now().astimezone(tz)

    weather_dict = {'city': [],
                'country': [],
                'forecast_time': [],
                'outlook': [],
                'detailed_outlook': [],
                'temperature': [],
                'temperature_feels_like': [],
                'clouds': [],
                'rain': [],
                'snow': [],
                'wind_speed': [],
                'wind_deg': [],
                'humidity': [],
                'pressure': [],
                'information_retrieved_at': []}

    for city in cities:
        url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")
        response = requests.get(url)
        json = response.json()
        
        for i in json['list']:
            weather_dict['city'].append(json['city']['name'])
            weather_dict['country'].append(json['city']['country'])
            weather_dict['forecast_time'].append(i['dt_txt'])
            weather_dict['outlook'].append(i['weather'][0]['main'])
            weather_dict['detailed_outlook'].append(i['weather'][0]['description'])
            weather_dict['temperature'].append(i['main']['temp'])
            weather_dict['temperature_feels_like'].append(i['main']['feels_like'])
            weather_dict['clouds'].append(i['clouds']['all'])
            try:
                weather_dict['rain'].append(i['rain']['3h'])
            except:
                weather_dict['rain'].append('0')
            try:
                weather_dict['snow'].append(i['snow']['3h'])
            except:
                weather_dict['snow'].append('0')
            weather_dict['wind_speed'].append(i['wind']['speed'])
            weather_dict['wind_deg'].append(i['wind']['deg'])
            weather_dict['humidity'].append(i['main']['humidity'])
            weather_dict['pressure'].append(i['main']['pressure'])
            weather_dict['information_retrieved_at'].append(now.strftime("%d/%m/%Y %H:%M:%S"))

    return pd.DataFrame(weather_dict)

In [10]:
get_weather_loop(['Berlin','Hamburg' ,'London', 'Manchester'])

,city,country,forecast_time,outlook,detailed_outlook,temperature,temperature_feels_like,clouds,rain,snow,wind_speed,wind_deg,humidity,pressure,information_retrieved_at
0,Berlin,DE,2022-07-08 12:00:00,Clouds,broken clouds,20.43,20.17,75,0,0,6.58,308,63,1008,08/07/2022 13:07:17
1,Berlin,DE,2022-07-08 15:00:00,Clouds,broken clouds,19.83,19.57,83,0,0,4.67,309,65,1014,08/07/2022 13:07:17
2,Berlin,DE,2022-07-08 18:00:00,Clouds,broken clouds,19.90,19.54,80,0,0,4.20,264,61,1019,08/07/2022 13:07:17
3,Berlin,DE,2022-07-08 21:00:00,Clouds,scattered clouds,16.46,15.88,49,0,0,3.47,295,66,1024,08/07/2022 13:07:17
4,Berlin,DE,2022-07-09 00:00:00,Clouds,scattered clouds,13.67,13.13,39,0,0,3.68,290,78,1024,08/07/2022 13:07:17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,Manchester,GB,2022-07-12 21:00:00,Clouds,overcast clouds,19.87,19.53,99,0,0,1.14,266,62,1016,08/07/2022 13:07:17
156,Manchester,GB,2022-07-13 00:00:00,Clouds,overcast clouds,19.21,19.04,99,0,0,1.67,273,71,1016,08/07/2022 13:07:17
157,Manchester,GB,2022-07-13 03:00:00,Clouds,overcast clouds,17.83,18.02,96,0,0,2.57,267,90,1016,08/07/2022 13:07:17
158,Manchester,GB,2022-07-13 06:00:00,Clouds,overcast clouds,17.84,17.95,98,0,0,2.91,269,87,1016,08/07/2022 13:07:17
